# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [0]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re

In [0]:
# Importando pacotes para treino e validação do modelo

from sklearn.feature_extraction.text import TfidfVectorizer
# um classificador linear que utiliza o Gradiente Descendente Estocástico como método de treino. 
# Por padrão, utiliza o estimador SVM.
from sklearn.linear_model import SGDClassifier
# Uma rede neural Perceptron Multicamadas
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [24]:
#Carregar os dados para o google colab

uploaded = files.upload()

Saving Dados Gerais Datasets.txt to Dados Gerais Datasets.txt


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [26]:
# Exemplos dos dados treino

df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA


In [27]:
# Exemplos dos dados treino teste

df2.head(2)

,0,DESCRIÇÃO PARCEIRO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952


In [28]:
# Verificação de nulos

df.isna().sum()

DESCRIÇÃO PARCEIRO    0
SUB-CATEGORIA         0
CATEGORIA             0
DEPARTAMENTO          0
dtype: int64

In [29]:
# Tamanho do dataset de treino (Vamos usar para treino e teste afim de validar os dados)

df.shape

(22009, 4)

In [30]:
# Quantidade das SUB-CATEGORIA, das CATEGORIA e das DEPARTAMENTO

classifica = df[['DEPARTAMENTO','CATEGORIA', 'SUB-CATEGORIA']]
classifica.nunique()

DEPARTAMENTO       53
CATEGORIA         332
SUB-CATEGORIA    1968
dtype: int64

In [31]:
# Tamanho das descrições

np.sort(df['DESCRIÇÃO PARCEIRO'].str.len().unique())

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 69, 70, 71, 74])

In [32]:
# Tamanho médio das descrições

df['DESCRIÇÃO PARCEIRO'].str.len().unique().mean()

37.292307692307695

### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.

df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')

In [34]:
df.head(10)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C/1
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S/GAS
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA,PAPELARIA,FITA DUPLA FACE C/SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT
5,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE 104G,LANCHINHO,BISCOITO,MERCEARIA DOCE,BISC LANCH BAUDUCCO RECHEADINHO CHOCOLATE
6,VINHO FRANCES CHEVALIER CHABLIS BCO 750ML,VINHO FRANCÊS,VINHO,BEBIDAS ALCOÓLICAS,VINHO FRANCES CHEVALIER CHABLIS BCO
7,DES ROLL REXONA INVISIBLE 50ML,ROLL ON COM PERFUME,DESODORANTE MASCULINO,CUIDADOS COM O CORPO,DES ROLL REXONA INVISIBLE
8,"SORVETE NOBRELLI 1,5L PT FLOCOS",POTE TRADICIONAL,SORVETE,CONGELADOS DOCES,SORVETE NOBRELLI
9,NOZ MOSCADA KONSABOR 10G,NOZ MOSCADA,CONDIMENTO NATURAL,MERCEARIA SALGADA,NOZ MOSCADA KONSABOR


### Etapa 1 - Classificando DEPARTAMENTO

In [0]:
#Criando a nova para classificar DEPARTAMENTO

base = df

In [0]:
# Selecionando apenas o item a ser classificado e o target do DF principal

base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']

In [0]:
# Transformando a descrição em vetor

stop_words=['de','da','do','com','para','c/','kg','cx','pc','gr','un','ml','pct','gfa','jg','p/','la']
vetor = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x = vetor.fit_transform(base_data)

In [0]:
# Dividindo dataset em treino e teste

x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size= 0.05, random_state=27)

In [0]:
# Aqui nós configuramos o modelo para treinar o classificador

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(200, ), random_state=1, verbose=True, max_iter=400)

In [40]:
# Mostrar o tempo de processamento
%%time

# Executar o treinamento
clf.fit(x_train, y_train)

CPU times: user 2min 40s, sys: 1min 1s, total: 3min 42s
Wall time: 2min 6s


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [41]:
# Avaliando a performance com predição

predicted = clf.predict(x_test)

print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       1.00      0.79      0.88        14
                 AÇOUGUE       0.89      0.80      0.84        20
      BEBIDAS ALCOÓLICAS       1.00      0.98      0.99        61
  BEBIDAS NÃO ALCOÓLICAS       0.96      0.84      0.90        32
               BRINQUEDO       1.00      0.80      0.89         5
                 CALÇADO       0.97      0.94      0.95        32
      CAMA, MESA E BANHO       1.00      1.00      1.00        25
        CONGELADOS DOCES       1.00      0.89      0.94         9
     CONGELADOS SALGADOS       0.86      0.67      0.75         9
    CUIDADOS COM A BARBA       1.00      1.00      1.00         1
     CUIDADOS COM A CASA       1.00      1.00      1.00        24
  CUIDADOS COM A COZINHA       1.00      0.83      0.91        12
    CUIDADOS COM A ROUPA       0.94      1.00      0.97        17
 CUIDADOS COM MÃOS E PÉS       1.00      

In [0]:
# Predição em novos dados com o DF  de teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','  ']
for x in pontuacao:
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')

base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)

In [0]:
# Adicionando a coluna para o novo DF

df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos

In [44]:
# Novos Dados de teste incluindo o departamento

df_dep.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO,MERCEARIA SALGADA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/,DECORAÇÃO


### Etapa 2 - Classificando CATEGORIA

In [0]:
# Criando a nova base para classificar CATEGORIA
base2 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data2 = base2['DEPARTAMENTO']+' '+base2['DESC_AJUSTADA']
base_target2 = base2['CATEGORIA']

In [0]:
# Transformando a descrição em vetor

stop_words=['de','da','do','com','para','c/','kg','cx','pc','gr','un','ml','pct','gfa','jg','p/','la']
vetor2 = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x2 = vetor2.fit_transform(base_data2)

In [0]:
# Dividindo dataset em treino e teste

x_train2, x_test2, y_train2, y_test2 = train_test_split(vetor_x2, base_target2, test_size=0.05, random_state=27)

In [48]:
# Mostrar o tempo de processamento
%%time

# Aqui nós configuramos o modelo para treinar o classificador
clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(300, ), random_state=1, verbose=True, max_iter=400)

# Executar o treinamento
clf2.fit(x_train2, y_train2)

CPU times: user 6min 57s, sys: 1min 32s, total: 8min 29s
Wall time: 5min 1s


In [49]:
# Avaliando a performance com predição

predicted2 = clf2.predict(x_test2)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test2, predicted2))


###Indicadores Classificação Categoria###

                                   precision    recall  f1-score   support

                       ABSORVENTE       1.00      1.00      1.00         5
              ACESSÓRIOS DE BANHO       1.00      1.00      1.00         2
         ACESSÓRIOS DE LAVANDERIA       1.00      1.00      1.00         4
         ACESSÓRIOS DE MANUTENÇÃO       0.00      0.00      0.00         1
                ACESSÓRIOS ELETRO       1.00      1.00      1.00         3
              ACESSÓRIOS MÃO E PÉ       1.00      1.00      1.00         6
           ACESSÓRIOS PARA CABELO       1.00      1.00      1.00         9
                   ACESSÓRIOS PET       1.00      1.00      1.00         1
                         ADOÇANTE       1.00      1.00      1.00         3
               AGUARDENTE/CACHAÇA       1.00      1.00      1.00         1
                          ALGODÃO       1.00      1.00      1.00         2
               ALIMENTO PARA CÃES       1.00      1.00 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados utilizando o df_dep
base_novos2 = df_dep['DEPARTAMENTO']+' '+ df_dep['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n2 = vetor2.transform(base_novos2)

#Previsão dos novos dados
predicted_novos2 = clf2.predict(vetor_n2)

In [0]:
# Adicionando a coluna no novo DF

df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos2

In [52]:
# Novos Dados de teste incluindo o departamento

df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO,MERCEARIA SALGADA,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/,DECORAÇÃO,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova para classificar CATEGORIA retirando apenas números

df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')

base3 = df

# Selecionando apenas o item a ser classificado e o target do DF principal

base_data3 = base3['DEPARTAMENTO']+' '+base3['CATEGORIA']+' '+base3['DESC_AJUSTADA']
base_target3 = base3['SUB-CATEGORIA']

In [0]:
# Transformando a descrição em vetor

stop_words=['de','da','do','com','para','c/','kg','cx','pc','gr','un','ml','pct','gfa','jg','p/','la']
vetor3 = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x3 = vetor3.fit_transform(base_data3)

In [0]:
# Dividindo dataset em treino e teste

x_train3, x_test3, y_train3, y_test3 = train_test_split(vetor_x3, base_target3, test_size=0.05, random_state=27)

In [56]:
# Mostrar o tempo de processamento
%%time

# Aqui nós treinamos o classificador
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(500, ), random_state=1, verbose=True, max_iter=400)

# Executar o treinamento
clf3.fit(x_train3, y_train3)

CPU times: user 1h 26min 24s, sys: 3min 51s, total: 1h 30min 16s
Wall time: 50min 13s


In [57]:
# Avaliando a performance com predição

predicted3 = clf3.predict(x_test3)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test3, predicted3))


###Indicadores Classificação Categoria###

                                          precision    recall  f1-score   support

                               ABOBRINHA       1.00      1.00      1.00         1
                                 ABÓBORA       1.00      1.00      1.00         1
                               ACENDEDOR       1.00      1.00      1.00         1
                                 ACEROLA       1.00      1.00      1.00         1
                    ACESSÓRIOS MANGUEIRA       0.50      1.00      0.67         1
           ACESSÓRIOS MAQUIAGEM - PINCEL       1.00      1.00      1.00         1
            ACESSÓRIOS MAQUIAGEM - PINÇA       1.00      1.00      1.00         1
                        ACESÓRIOS PANELA       0.00      0.00      0.00         1
                  AEROSOL ANTIBACTERIANO       0.00      0.00      0.00         0
                         AEROSOL BARATAS       0.00      0.00      0.00         1
                       AEROSOL CLAREADOR       0.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados
df_cat['DESC_AJUSTADA'] = df_cat['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','  ']
for x in pontuacao:
  df_cat['DESC_AJUSTADA'] = df_cat['DESC_AJUSTADA'].str.replace(x, ' ')

base_novos3 = df_cat['DEPARTAMENTO']+' '+df_cat['CATEGORIA']+' '+df_cat['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n3 = vetor3.transform(base_novos3)

#Previsão dos novos dados
predicted_novos3 = clf3.predict(vetor_n3)

In [0]:
# Adicionando a coluna

df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos3

In [60]:
# Novos Dados de teste incluindo o departamento

df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO G,MERCEARIA SALGADA,SAL,SAL GROSSO DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/ DR/PR/BC R,DECORAÇÃO,ENFEITES DE NATAL,BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE,CHÁ,BRANCO
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ACESSÓRIOS MÃO E PÉ,LIXA PÉS
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,PIZZA,MARGUERITA
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK M,DESCARTÁVEIS,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S XXH,LAVANDERIA,ORGANIZADOR,CESTO ORGANIZADOR
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI GR OVOS ESPAGUETE,MERCEARIA SALGADA,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON ML,CUIDADOS COM O CABELO,CONDICIONADOR,TRADICIONAL


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')

In [62]:
from joblib import dump, load
dump(clf, 'modelo_etapa1.joblib') 
dump(clf2, 'modelo_etapa2.joblib') 
dump(clf3, 'modelo_etapa3.joblib') 
#clf = load('modelo_etapas.joblib')

['modelo_etapa3.joblib']

### Referências

https://medium.com/luisfredgs/classificando-textos-com-machine-learning-e054ca7bf4e0

https://www.alura.com.br/artigos/classificando-textos-com-python